In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import cv2
import yaml
from pypylon import pylon 

from IPython.display import Image, display, clear_output

%matplotlib inline

In [ ]:
# Pypylon get camera by serial number
serial_number = '22716152'
info = None
for i in pylon.TlFactory.GetInstance().EnumerateDevices():
    if i.GetSerialNumber() == serial_number:
        info = i
        break
else:
    print('Camera with {} serial number not found'.format(serial_number))

# VERY IMPORTANT STEP! To use Basler PyPylon OpenCV viewer you have to call .Open() method on you camera
if info is not None:
    camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateDevice(info)) 
    camera.Open()

In [ ]:
# List of features to create wigets
features = [
    {
        "name": "GainRaw",
        "type": "int"
    },
    {
        "name": "ExposureTimeRaw",
        "type": "int",
        "max": 35000,
        "min": 24,
        "step": 140,
        "value": 3000
    },
    {
        "name": "Height",
        "type": "int_text",
        "max": 1200,
        "min": 100,
        "step": "5"
    },
    {
        "name": "Width",
        "type": "int_text",
        "max": 1920,
        "min": 100,
        "step": "5"
    },
    {
        "name": "AcquisitionFrameRateEnable",
        "type": "bool"
    },
    {
        "name": "AcquisitionFrameRateAbs",
        "type": "int",
        "max": 60,
        "min": 1,
        "value": 30
    }
]

In [ ]:
i = 0
def impro(img):
    global i
    cv2.namedWindow('1', cv2.WINDOW_NORMAL | cv2.WINDOW_GUI_NORMAL)
    cv2.resizeWindow('1', 1080, 720)
    cv2.imshow("1",img)
    i += 1
    if i % 30 == 0:
        clear_output(wait=True)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        hist = cv2.calcHist([gray],[0],None,[256],[0,256])
        plt.hist(gray.ravel(),256,[0,256])
        plt.title('Histogram for gray scale picture')
        plt.show()

In [ ]:
from pypylon_opencv_viewer import BaslerOpenCVViewer
    
viewer = BaslerOpenCVViewer(camera)
viewer.set_features(features)
viewer.set_impro_function(impro)
viewer.run_interaction_continuous_shot()